# Reproducing: Modelling Sub-Exponential Growth of COVID-19 in Australia

Original paper: Associate Professor Nigel Marks - Saturday, 28 March 2020

### STATUS: Work-in-progress

## Imports

In [ ]:
# !pip install lmfit

In [81]:
import pandas as pd
import numpy as np
import matplotlib as plt
from os import listdir
from datetime import timedelta

%matplotlib inline

# Import and suppress warnings

In [82]:
from numpy import exp, sin
from lmfit import minimize, Parameters

## Data

### Countries

In [ ]:
# !unzip covid-19_data_NMarks_4Apr2020.zip

```Archive:  covid-19_data_NMarks_4Apr2020.zip
  inflating: au/data.txt             
  inflating: ca/data.txt             
  inflating: ch/data.txt             
  inflating: es/data.txt             
  inflating: fr/data.txt             
  inflating: it/data.txt             
  inflating: pt/data.txt             
  inflating: uk/data.txt             
  inflating: us/data.txt  ```

In [ ]:
# !ls data

```au  ca	ch  es	fr  it	pt  uk	us```

In [ ]:
# !head -n 10 data/au/data.txt

```
3	40	
4	51	
5   59      
6	63	
7	73	
8	80	
9	92
10	112	
11	127	
12	157 ```

In [ ]:
countries = [subdir for subdir in listdir('data/') if len(subdir) <= 2]

In [ ]:
countries

In [ ]:
data = []
for subdir in countries:
    data.append([subdir, pd.read_table('data/' + subdir + '/data.txt', header=None, delim_whitespace=True, names=['nDay', 'nPeopleInfected'])])

# data[country][data_frame[nDay][nPeopleInfected]]

In [ ]:
BASE_DATE = '1-Mar-2020'    # TODO: Need to verify the actual date

In [ ]:
# The following is a bit clumsy - should be an array/column method to do this without looping

for iCountry, subdir in enumerate(listdir('data/')):
    data[iCountry][1]['Date'] = pd.to_datetime(BASE_DATE)
    for idx, row in data[iCountry][1].iterrows():
        data[iCountry][1]['Date'][idx] = data[iCountry][1]['Date'][idx] + pd.DateOffset(days=idx)

In [ ]:
# Australian data

# TODO - use Seaborn or Bokeh for better (interactive) charts

data[0][1].plot(x='Date', y='nPeopleInfected')

In [ ]:
data[0][1].head(5)

### Australian data by State/Territory

In [ ]:
!head -n 10 data/covid-19_nsw_data.txt

In [ ]:
AusStates_df = pd.read_table('data/covid-19_nsw_data.txt', delim_whitespace=True)

In [ ]:
AusStates_df['Date'] = pd.to_datetime(BASE_DATE)

In [ ]:
for idx, row in AusStates_df.iterrows():
        AusStates_df['Date'][idx] = AusStates_df['Date'][idx] + pd.DateOffset(days=idx)

In [ ]:
AusStates_df

In [ ]:
# TODO: Allow for sensitivity analysis and SMA 2-3 day to smooth reporting biases

The data is sourced via the author and also available from: https://www.health.gov.au/news/health-alerts/novel-coronavirus-2019-ncov-health-alert/coronavirus-covid-19-current-situation-and-case-numbers and
https://covid19data.com.au.

This site https://www.theguardian.com/australia-news/datablog/ng-interactive/2020/mar/28/coronavirus-australia-map-cases-numbers-stats-how-many-cases-of-covid-19-nsw-maps-victoria-live-data-qld-sa-wa-tas-latest-statistics is another possible source.

Could potentially manually scrape data from here: https://www.theguardian.com/australia-news/datablog/ng-interactive/2020/apr/01/coronavirus-cases-in-australia-map-confirmed-numbers-stats-how-many-cases-of-covid-19-nsw-maps-victoria-live-data-qld-sa-wa-tas-nt-act-latest-statistics-update

## Theory

When exponential growth is occurring, the system is governed by the differential equation

\begin{equation*}
\frac{dN}{dt} = kN
\end{equation*}

for which the solution can immediately be written in the form

\begin{equation*}
N(t) = N_0 \cdot 2^{t/t_{double}}
\end{equation*}

where $N$ is the number of infected individuals, $t$ is time measured in days, $N_0$ is the value of $N$ at time $t=0$, and $t_{double}$ is the time taken for the number of infections to double. 

Across Australia, $t_{double}$ is typically around 3 days. For much of March 2020, the above expression provided a good fit to the data for Australia as a whole, as well as for most jurisdictions.

However, over the last few days, some of the data began to diverge from a pure exponential, especially in Victoria. Consider the graph below, which attempts to fit a single exponential to the data. Even though the numbers are increasing sharply, as shown in the graph on the left, the behaviour is not purely exponential, as the data cannot be described by a straight line on a log-linear graph. Something else must be going on.

In [ ]:
# TODO: plots x 2 - # cases VIC vs date (linear/linear) and # cases VIC vs date (log/linear)

One possible explanation is that the social distancing measures and other actions introduced by the government are starting to be reflected in official figures. 
Due to the incubation period of COVID-19, changes in stimuli take up to 2 weeks to be clearly seen in the number of reported cases. 
This reflects the phase of the disease known as community transmission, where the virus jumps from person-to-person. 
Modifying community behaviour affects the transmission rate, but time must pass to observe the effect.

Since the Australian government does not appear to be following the eradication strategy of the NZ government, it is reasonable to presume that their approach will be one of herd immunity, where the goal is for at least 60% of the population to develop immunity via controlled exposure to the virus. 
For this strategy to work, it is vital that the hospital system is not overwhelmed, in particular the capacity of intensive care units, or ICUs.

The hospital system can only process sick patients at a fixed rate, and hence the herd immunity strategy relies on the rate of new cases being constant. In other words, even though the rate of change of $N$ is exponential in the early phases of the epidemic, it must be brought under control via measures such as social distancing and varying degrees of lockdown, so that in the near future one has the situation

\begin{equation*}
\frac{dN}{dt} = \text{constant}
\end{equation*}

where the specific value of the constant reflects the capacity of the hospital system. Note that this is a function of the length of time that sick individuals need high-level care. For example, a patient admitted to an ICU will typically require 10 days of care.

To develop a simple model of the system, we consider that exponential growth is present until some time $t_1$, after which control measures are introduced so that eventually $dN/dt$ becomes constant. 
In the Australian context, these measures are being introduced gradually, so let’s assume that the rate of new cases gradually reduces from a rate $R_1$ at time $t_1$ to a constant final rate $f \times R_1$, where $f$ is a number between 0 and 1, perhaps around 0.5. 
The simplest expression is that of a decaying exponential, which implies that $dN/dt$ has the form

\begin{equation*}
\frac{dN}{dt} = f R_1 + (R_1 - f R_1) \exp \left[ -0.693 \frac{t-t_1}{t_{\text{half}}} \right]
\end{equation*}

where $t_{\text{half}}$ controls the rate at which $dN/dt$ is adjusted from $R_1$ to $f R_1$. This situation is shown schematically in the left-hand graph below. Up until time $t_1$ the number of new cases each day (i.e. $dN/dt$) increases exponentially, but after measures are introduced, $dN/dt$ shifts towards a constant value at rate controlled by $t_{\text{half}}$. In the transition region between $t_1$ and $t_1 + 5 t_{\text{half}}$ the growth rate is sub-exponential, while for $t > t_1 + 5 t_{\text{half}}$ we have $dN/dt$ equal to a constant and hence the total number of cases $N$ increases linearly with time, as shown in the right-hand graph. We thus have three regions: (i) exponential, (ii) transition / sub-exponential, and (iii) steady-state / linear.

In [ ]:
# TODO: Create charts

Using this simple model we can write the following formula for $dN/dt$,

TODO: Continue from here (need to use eqnarray environment)

\begin{equation*}
\frac{dN}{dt} = \frac{1}{t_{double}} N_0 \times 2^{t/t_{double}}  t < t_1 
\end{equation*}

where the rate $R_1$ at time $t_1$ is given by

In [ ]:
# TODO: Add equation

Integrating the expression above to obtain $N(t)$ yields the expression:

TODO: Insert equation

giving us a five-parameter model to analyse our Victoria figures (or any other data set), where the unknowns are $N_0$, $t_{\text{double}}$, $t_1$, $f$ and $t_{\text{half}}$. Initial testing showed that $t_{\text{half}}$ was associated with large uncertainties, and hence its value was fixed at 3 days. By making this choice, a robust fit involving just four parameters is obtained. When further data becomes available it will be possible to relax this constraint and fit all five values.

*Question: what (non-linear) fitting technique was used?*

Try LMFit - https://lmfit.github.io/lmfit-py/intro.html

In [84]:
params = Parameters()
params.add('N_0', value=10, vary=False)
params.add('t_double', value=0.007, min=0.0)
params.add('t_1', value=0.2)
params.add('f', value=3.0, max=10)
params.add('t_half')    # fixed at 3 days TODO: try relaxing

In [ ]:
# TODO: Define the residual function


Applying this four-parameter model to the Australia data using the gnuplot package produces an excellent fit, suggesting that the assumptions are reasonable. A clear transition to sub-exponential behaviour appears around March 21st for VIC, WA and ACT, while for NSW this occurs a couple of days later. The value of $f$ is around 0.6 across multiple states. The plateau values reveal the steady-state case that needs to be accommodated by the hospital system. Keeping in mind that additional data will improve accuracy, estimates of the steady state number of new cases are 150 per day in NSW, 50 per day in Victoria, 20 per day in WA and 5 per day in the ACT. Overseas statistics suggest around 20% of these cases will require hospitalization and 5% will need intensive care.

In [ ]:
# TODO: Create chart

As the graph at right shows, similar behavior is seen for other Australian states and Territories. In the case of Tasmania and the Northern Territory, onset of infection was delayed and growth is still in the exponential phase. As further data is collected it will become possible to estimate $t_1$ and hence the final steady-state rate.

In [ ]:
# TODO: Create chart

## Summary

This analysis shows that the COVID-19 epidemic in Australia is being contained and is no longer increasing exponentially in most jurisdictions. This is consistent with control measures producing a constant infection rate.

In [80]:
## Assumptions (and sensistivity to these assumptions)

## Appendix: LMFit test example

In [83]:
x = np.linspace(0, 25, 201)

In [73]:
datar = 1.0

In [74]:
eps_data = 0.1

In [76]:
def residual(params, x, datar, eps_data):
    amp = params['amp']
    phaseshift = params['phase']
    freq = params['frequency']
    decay = params['decay']
    model = amp * sin(x*freq + phaseshift) * exp(-x*x*decay)
    return (datar-model) / eps_data

In [77]:
params = Parameters()
params.add('amp', value=10, vary=False)
params.add('decay', value=0.007, min=0.0)
params.add('phase', value=0.2)
params.add('frequency', value=3.0, max=10)

In [78]:
out = minimize(residual, params, args=(x, datar, eps_data))

In [79]:
out